# Simulating User

In HybridAGI, the User is simulated by default because being able to run a graph program without human input is required to optimize or finetune any interactive program. Many other frameworks to avoid this constraint just removed the possibility to interact with the user during the execution of a plan. But for us, with a background in Human-Human & Human-Robot collaboration, speaking is a goal-directed action that happen in a plan (or program in our case).

In HybridAGI, we made 2 different tools to interact with the User:

- `Speak`: This tool send a message to the user, the message is being stored into the chat history and the last message is given as final answer to the system. 
- `AskUser`: This tool ask a question to the user, both question and answer are being stored into the chat history, when the tool is simulated (by default) the answer is simulated by LLM based on the user profile and chat history.

In the following graph program, we are going to disambiguate the Objective of the program, to spice things up, we are going to make 2 programs and use sub-program calls.

In [1]:
import dspy
import hybridagi.core.graph_program as gp
from hybridagi.core.datatypes import (
    AgentState,
    Query,
)
from hybridagi.memory.integration.local import LocalProgramMemory
from hybridagi.modules.agents.tools import (
    SpeakTool,
    AskUserTool,
    UpdateObjectiveTool,
)
from hybridagi.modules.agents import GraphInterpreterAgent

clarify_objective = gp.GraphProgram(
    name = "clarify_objective",
    description = "Clarify the objective by asking question to the user",
)

clarify_objective.add(gp.Decision(
    id = "is_anything_unclear",
    purpose = "Check if the question is unclear",
    question = "Is the Objective's question still unclear?",
))

clarify_objective.add(gp.Action(
    id = "ask_question",
    purpose = "Ask question to clarify the Objective",
    tool = "AskUser",
    prompt = "Pick one question to clarify the Objective's question",
))

clarify_objective.add(gp.Action(
    id = "refine_objective",
    purpose = "Refine the question",
    tool = "UpdateObjective",
    prompt = "Refine the Objective's question by another question",
))

clarify_objective.connect("start", "is_anything_unclear")
clarify_objective.connect("ask_question", "refine_objective")
clarify_objective.connect("is_anything_unclear", "ask_question", label="Clarify")
clarify_objective.connect("is_anything_unclear", "end", label="Answer")
clarify_objective.connect("refine_objective", "end")

clarify_objective.build()

main = gp.GraphProgram(
    name="main",
    description="The main program",
)

main.add(gp.Program(
    id = "clarify_objective",
    purpose = "Clarify the Objective if needed",
    program = "clarify_objective",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer the Objective's question in a helpfull way",
    tool = "Speak",
    prompt = "Please answer the Objective's question",
))

main.connect("start", "clarify_objective")
main.connect("clarify_objective", "answer")
main.connect("answer", "end")

main.build()

program_memory = LocalProgramMemory(index_name="test")

program_memory.update(clarify_objective)
program_memory.update(main)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
        speak_func = None, # the callable to call outside the simulation (default None)
        simulated = True, # Weither or not to simulate the tool (default True)
    ),
    AskUserTool(
        agent_state = agent_state,
        ask_user_func = None, # the callable to call outside the simulation (default None)
        simulated = True, # Weither or not to simulate the tool (default True)
    ),
    UpdateObjectiveTool(
        agent_state = agent_state,
    )
]

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n", "\n---", "\n\nContext:"])
dspy.configure(lm=lm)

result = agent(Query(query="What is the meaning of life?"))

print(result.final_answer)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Step 0 ---
Call Program: main
Program Purpose: What is the meaning of life?
--- Step 1 ---
Call Program: clarify_objective
Program Purpose: Clarify the Objective if needed
--- Step 2 ---
Decision Purpose: Check if the question is unclear
Decision Question: Is the Objective's question still unclear?
Decision: CLARIFY
--- Step 3 ---
Action Purpose: Ask question to clarify the Objective
Action: {
  "question": "What specific aspect or perspective of 'the meaning of life' are you interested in?",
  "answer": "Curiosity about purpose or existence."
}
--- Step 4 ---
Action Purpose: Refine the question
Action: {
  "new_objective": "To explore and understand the concept of purpose or existence in relation to 'the meaning of life'."
}
--- Step 5 ---
End Program: clarify_objective
--- Step 6 ---
Action Purpose: Answer the Objective's question in a helpfull way
Action: {
  "message": "Hello! I have been designed to assist you with your questions. You asked about exploring and understanding th